In [77]:
from config import gkey
from config import weatherkey
from citipy import citipy
import numpy as np
import pandas as pd
import requests
import json
from pprint import pprint

In [78]:
# Build Cities Table 
# define empty lists
citylist = []
latlist = []
longlist = []
countrycodelist = []

# define length of script
MaxCities = 5


# create dataframe 
City_df = pd.DataFrame()
header_list = ['Lattitude','Longitude']
City_df = City_df.reindex(columns = header_list)      

# lats and longs
for cities in range(1,(MaxCities +1)):
    cities_lat = round(np.random.uniform(-90, 90),4)
    latlist.append(cities_lat)
    cities_long = round(np.random.uniform(-180, 180),4)
    longlist.append(cities_long)

# append Lattitude and Longitude to DF
City_df['Lattitude'] = pd.Series(latlist)
City_df['Longitude'] = pd.Series(longlist)


# Get Cities from PyCities
for i in range(0,len(City_df['Lattitude'])):
    lat = City_df.iloc[i]['Lattitude']
    long = City_df.iloc[i]['Longitude']
    cityname = citipy.nearest_city(lat,long).city_name.title()
    citycountry = citipy.nearest_city(lat,long).country_code.upper()
#     print(f'{lat} {long} {cityname} {citycountry}')
    citylist.append(cityname)
    countrycodelist.append(citycountry)

# Append Cities to DF
City_df.insert(0,'City',citylist)
City_df.insert(1, 'Country', countrycodelist)
City_df

,City,Country,Lattitude,Longitude
0,Mar Del Plata,AR,-44.9134,-52.5285
1,Barentsburg,SJ,81.7127,10.1912
2,Atuona,PF,-4.9927,-145.9087
3,Ushuaia,AR,-81.0224,-32.3609
4,Hilo,US,14.2829,-159.0300


In [79]:
# Edit Dataframe and add columns

City_df['Temperature C'] = ""
City_df['Temperature F'] = ""
City_df['Revised Lattitude'] = ""
City_df['Revised Longitude'] = ""
City_df['Humidity %'] = ""
City_df['Cloudiness %'] = ""
City_df['Wind Speed (mph)'] = ""
City_df

,City,Country,Lattitude,Longitude,Temperature C,Temperature F,Revised Lattitude,Revised Longitude
0,Mar Del Plata,AR,-44.9134,-52.5285,,,,
1,Barentsburg,SJ,81.7127,10.1912,,,,
2,Atuona,PF,-4.9927,-145.9087,,,,
3,Ushuaia,AR,-81.0224,-32.3609,,,,
4,Hilo,US,14.2829,-159.0300,,,,


In [89]:
# api.openweathermap.org/data/2.5/weather?q=London,uk&appid={API key}

# build URL
base_url = 'http://api.openweathermap.org/data/2.5/weather?q='
# city = "Sao Filipe"
# country = 'CV'
comma = ','
andsign = '&'


# API GET & JSON
for index, row in City_df.iterrows():
    city = row['City']
    country = row['Country']
    full_url = base_url + city + comma + country + '&appid=' + weatherkey
    response = requests.get(full_url) 
    parsed_response = response.json()
    try:
        City_df.at[index,'Temperature C'] = (parsed_response["main"]["temp_max"] - 273)
        City_df.at[index,'Temperature F'] = (parsed_response["main"]["temp_max"] * 1.8 - 457.6)
        City_df.at[index,'Revised Lattitude'] = parsed_response["coord"]["lat"]
        City_df.at[index,'Revised Longitude'] = parsed_response["coord"]["lon"]
        City_df.at[index,'Humidity %'] = parsed_response["main"]["humidity"]
        City_df.at[index,'Cloudiness %'] = parsed_response["clouds"]["all"]
        City_df.at[index,'Wind Speed (mph)'] = (parsed_response["wind"]["speed"] * 2.24)
        print('city found: ' + city + " ," + country)
    except:
        print('City ' + city + ' not found! skipping')
        next
#     print(city + " " + str(CityTemp))




city found: Mar Del Plata ,AR
City Barentsburg not found! skipping
city found: Atuona ,PF
city found: Ushuaia ,AR
city found: Hilo ,US


In [90]:
City_df

,City,Country,Lattitude,Longitude,Temperature C,Temperature F,Revised Lattitude,Revised Longitude,Humidity %,Cloudiness %,Wind Speed (mph)
0,Mar Del Plata,AR,-44.9134,-52.5285,17.37,65.066,-38.0023,-57.5575,82.0,0.0,11.5136
1,Barentsburg,SJ,81.7127,10.1912,,,,,NaN,NaN,NaN
2,Atuona,PF,-4.9927,-145.9087,26.64,81.752,-9.8,-139.033,76.0,85.0,12.9024
3,Ushuaia,AR,-81.0224,-32.3609,8.15,48.47,-54.8,-68.3,76.0,75.0,4.6144
4,Hilo,US,14.2829,-159.0300,19.15,68.27,19.7297,-155.09,82.0,75.0,4.4128


In [88]:
city = "Busselton"
country = 'AU'
full_url = base_url + city + comma + country + '&appid=' + weatherkey
response = requests.get(full_url) 
parsed_response = response.json()
# CityTemp = parsed_response["main"]["temp"]
pprint(parsed_response)


{'base': 'stations',
 'clouds': {'all': 72},
 'cod': 200,
 'coord': {'lat': -33.65, 'lon': 115.3333},
 'dt': 1616160761,
 'id': 2075265,
 'main': {'feels_like': 295.8,
          'humidity': 71,
          'pressure': 1017,
          'temp': 294.26,
          'temp_max': 294.26,
          'temp_min': 294.26},
 'name': 'Busselton',
 'sys': {'country': 'AU',
         'id': 2008303,
         'sunrise': 1616106055,
         'sunset': 1616149926,
         'type': 3},
 'timezone': 28800,
 'visibility': 10000,
 'weather': [{'description': 'broken clouds',
              'icon': '04n',
              'id': 803,
              'main': 'Clouds'}],
 'wind': {'deg': 45, 'gust': 1.79, 'speed': 0.45}}


In [ ]:
if 'Bluff' in City_df['City'].values:
    print('yep!')

